# AI FAQ 챗봇 만들기 with RAG

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import json

doc = json.load(open('data/faq_chatbot_data.json'))
doc , len(doc)

([{'question': '반품 정책이 어떻게 되나요?',
   'answer': '제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'},
  {'question': '배송은 얼마나 걸리나요?',
   'answer': '일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'},
  {'question': '회원가입 없이 주문할 수 있나요?',
   'answer': '네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'},
  {'question': '포인트 적립은 어떻게 되나요?',
   'answer': '구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'},
  {'question': '해외 배송이 가능한가요?',
   'answer': '일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'},
  {'question': '제품 보증 기간은 얼마나 되나요?',
   'answer': '제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'},
  {'question': 'A/S 신청은 어떻게 하나요?',
   'answer': '공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'},
  {'question': '재고가 없는 상품은 언제 다시 입고되나요?',
   'answer': '입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'},
  {'question': '구매 후 영수증을 받을 수 있나요?',
   'answer': '네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'},
  {'question': '다른 사람이 대신 주문을 수령할 수 있나요?',
   'answer': '네, 수령인을 지정할 수 있으며 신분증 확인 후 수령 가능합니다.'},
  {'question': '결제 방법에는 어떤 것이 있나요?'

In [22]:
# question 과 answer 붙이기
qa = []
for i in range(len(doc)):
    qa.append(doc[i]['question'] + ' ' + doc[i]['answer'])
    

In [27]:
doc = [Document(page_content=qa[i]) for i in range(len(qa))]
doc

[Document(metadata={}, page_content='반품 정책이 어떻게 되나요? 제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'),
 Document(metadata={}, page_content='배송은 얼마나 걸리나요? 일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'),
 Document(metadata={}, page_content='회원가입 없이 주문할 수 있나요? 네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'),
 Document(metadata={}, page_content='포인트 적립은 어떻게 되나요? 구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'),
 Document(metadata={}, page_content='해외 배송이 가능한가요? 일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'),
 Document(metadata={}, page_content='제품 보증 기간은 얼마나 되나요? 제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'),
 Document(metadata={}, page_content='A/S 신청은 어떻게 하나요? 공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'),
 Document(metadata={}, page_content='재고가 없는 상품은 언제 다시 입고되나요? 입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'),
 Document(metadata={}, page_content='구매 후 영수증을 받을 수 있나요? 네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'),
 Document(metadata={}, page_content='다른 사람이 대신 주문을 수령할 수 있나요? 네, 수령인을 지정할 수 있으며 신분증 확인 

In [28]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [32]:
vector_store = Chroma.from_documents(doc, embedding_model)

In [33]:
query = '해외 배송 가능 여부'

retrievals = vector_store.similarity_search_with_score(query)
retrievals

[(Document(id='9ef7642d-5060-4c91-b25e-ad2bce495d10', metadata={}, page_content='해외 배송이 가능한가요? 일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'),
  0.5525962114334106),
 (Document(id='ef5fd9ce-8c9e-4e62-83e2-64728e9342aa', metadata={}, page_content='해외 배송이 가능한가요? 일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'),
  0.5533106923103333),
 (Document(id='eec0daff-baf3-4030-a36b-ac72b9b9c084', metadata={}, page_content='배송 주소를 변경할 수 있나요? 출고 전이라면 고객센터를 통해 배송 주소 변경이 가능합니다.'),
  1.083909511566162),
 (Document(id='c66713b3-0ae9-4ace-a073-a3f85b84fd36', metadata={}, page_content='배송 주소를 변경할 수 있나요? 출고 전이라면 고객센터를 통해 배송 주소 변경이 가능합니다.'),
  1.08420729637146)]

In [34]:
# Retriever를 사용한 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k':3}
)

retriever_result = retriever.batch([query])
retriever_result

[[Document(id='9ef7642d-5060-4c91-b25e-ad2bce495d10', metadata={}, page_content='해외 배송이 가능한가요? 일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'),
  Document(id='ef5fd9ce-8c9e-4e62-83e2-64728e9342aa', metadata={}, page_content='해외 배송이 가능한가요? 일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'),
  Document(id='eec0daff-baf3-4030-a36b-ac72b9b9c084', metadata={}, page_content='배송 주소를 변경할 수 있나요? 출고 전이라면 고객센터를 통해 배송 주소 변경이 가능합니다.')]]

In [49]:
prompt = PromptTemplate(
    template='{query}에 맞는 답을 주어진 {context}에 맞게 알맞은 답을 주고 그 후 추가 질문에 대해 미리 예측해서 몇가지 질분에 대한 답도 보여줘.',
    input_variables={'query','context'}
)

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0
)

In [50]:
query = '해외 배송 가능한가?'
retrievals = retriever.batch([query])
context_text = '\n'.join([doc.page_content for doc in retrievals[0]])
chain = prompt | model

In [51]:
response = chain.invoke({'query':query, 'context':context_text}).content
response

'**해외 배송이 가능한가요?**  \n네, 일부 국가를 대상으로 해외 배송이 가능합니다. 배송비는 지역에 따라 상이할 수 있습니다.\n\n**배송은 얼마나 걸리나요?**  \n일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.\n\n---\n\n**추가 질문 예측 및 답변:**\n\n1. **어떤 국가로 배송이 가능한가요?**  \n   현재 해외 배송이 가능한 국가는 미국, 캐나다, 유럽 일부 국가 등입니다. 자세한 목록은 웹사이트에서 확인하실 수 있습니다.\n\n2. **배송비는 어떻게 계산되나요?**  \n   배송비는 주문하신 상품의 무게와 배송 지역에 따라 자동으로 계산됩니다. 결제 과정에서 확인하실 수 있습니다.\n\n3. **배송 추적은 어떻게 하나요?**  \n   배송이 시작되면 이메일로 발송된 추적 번호를 통해 배송 상태를 실시간으로 확인하실 수 있습니다.\n\n4. **반품은 어떻게 하나요?**  \n   해외 배송 상품의 반품은 수령 후 14일 이내에 요청하셔야 하며, 반품 절차에 대한 자세한 내용은 고객센터에 문의해 주시기 바랍니다.\n\n5. **관세는 어떻게 되나요?**  \n   해외 배송 시 발생할 수 있는 관세는 수령인 부담이며, 각 국가의 세관 규정에 따라 다를 수 있습니다. 자세한 사항은 해당 국가의 세관에 문의하시기 바랍니다.'

## 강사님 ver

In [52]:
# 데이터 로드
with open('./data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    faq_data = json.load(f)

In [53]:
# embedding 모델 생성 + vector store 생성
embedding_model = OpenAIEmbeddings()

vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='./chroma_db'
)

documents = [Document(page_content=item['question']+ " " + item['answer']) for item in faq_data]
vector_store.add_documents(documents)

['6d6c5b42-4b74-433e-92fe-9c2d067d0851',
 'b4eea25d-53c4-4be5-bcab-85e3b586dd96',
 '5efbfadc-54ab-4cdd-8855-9cd25994813d',
 'b0bd3522-fbe3-4458-bb82-e519c7e55bd0',
 '77f1564b-0a86-45c9-9a51-8f87f725dcb5',
 '9dd2f6af-e678-4ba6-92ff-eba9ab7abd6d',
 '5f2bf15b-26df-4a46-ab56-8e69a901c706',
 '08a082a8-0c18-4a7d-a9c4-b32913abc4cb',
 '83b08e5a-85ec-4c2d-b32e-cfb3a4783286',
 'f7f77dac-731c-444d-b037-fc356c96b91d',
 '2f441255-1d0e-49e8-b109-4da1dad4f6bf',
 '8ad38140-3a33-4289-aada-cfa9333705c8',
 'bc0773aa-a46d-4955-8c86-45f9eed6c659']

In [54]:
# llm 설정 (모델 생성)
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature=0.5
)

In [59]:
prompt = PromptTemplate(
    template="""
    다음 '질문'에 대해 '문서'의 내용만을 참고하여 '답변'을 생성해 주세요.
    질문 : {question}
    문서 : {context}
    답변 : 
    """,
    input_variables={'question','context'}
)

In [60]:
retriever = vector_store.as_retriever()

In [61]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type='stuff',
    retriever=retriever,
    chain_type_kwargs={'prompt':prompt}
)

In [ ]:
# 체인 설정


In [62]:
# 사용
qa_chain('해외 배송 가능해?')

{'query': '해외 배송 가능해?',
 'result': '해외 배송은 일부 국가를 대상으로 가능하며, 배송비는 지역에 따라 다를 수 있습니다.'}